In [1]:
from solve_LMI import solve_LMI
from solve_LP import solve_LP
import numpy as np
from scipy.linalg import toeplitz
from circulant_embedding import faster_A
from scipy.linalg import circulant
from scipy.sparse import diags, csr_array
from SCGAL import run_solver
import time

In [15]:
n = 1
# d = 10
max_runs = 100

ds = range(1, 6)
circ_emb_obj = np.zeros((max_runs, len(ds)))
circ_emb_time = np.zeros((max_runs, len(ds)))
LMI_obj = np.zeros((max_runs, len(ds)))
LMI_time = np.zeros((max_runs, len(ds)))
SCGAL_obj = np.zeros((max_runs, len(ds)))
SCGAL_time = np.zeros((max_runs, len(ds)))
cs = np.random.normal(size=(4*ds[-1], len(ds)))

# c = np.random.normal(size=4*d)

In [ ]:
# Circulant embedding
for i in range(len(ds)):
    d = ds[i]
    c = cs[:4*d, i]

    A = faster_A(n, d)
    A_0 = np.ones(np.shape(A)[0])
    print("C Norm", np.linalg.norm(c, ord=1))

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        tic = time.perf_counter()
        status_LP, optimal_value_LP, optimal_vars_LP = solve_LP(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        circ_emb_obj[i, num_iterations] = optimal_value_LP
        circ_emb_time[i, num_iterations] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)
        
        # print("LP Status:", status_LP)
        # print("LP Optimal Value:", optimal_value_LP)
        # print("LP Optimal Variables:", [var.value for var in optimal_vars_LP])

C Norm 4.047983204302768
1 0.00533475202973932
1 0.00907161901704967
1 0.012049441051203758
1 0.014905249001458287
1 0.019321164989378303
1 0.02407564502209425
1 0.027582636976148933
1 0.03182954894145951
1 0.036602803971618414
1 0.04078767297323793
1 0.0442746399785392
1 0.047154292929917574
1 0.05086040194146335
1 0.054064471914898604
1 0.05677157989703119
1 0.05944504193030298
1 0.06206126190954819
1 0.0646322799148038
1 0.06715125287882984
1 0.06961112888529897
1 0.07195878389757127
1 0.07442264893325046
1 0.07685965194832534
1 0.0791631099418737
1 0.08175167493754998
1 0.0846761878929101
1 0.08790984191000462
1 0.0906799619551748
1 0.0933528199675493
1 0.09596155298640952
1 0.09855438099475577
1 0.10157568397698924
1 0.10434362100204453
1 0.10701478098053485
1 0.1097767079481855
1 0.11251422791974619
1 0.11486655293265358
1 0.11706890090135857
1 0.11941210686927661
1 0.12194518587784842
1 0.12435200187610462
1 0.12672693887725472
1 0.12915312789846212
1 0.13141946191899478
1 0.133

In [ ]:
# LMI formulation with toeplitz constraints
col = np.zeros(2*d + 1, dtype=complex)
col[0] = 1
A_0 = toeplitz(col)

A = []

for m in range(1, 2*d + 1):
    col = np.zeros(2*d + 1, dtype=complex)
    col[m] = 1
    mx = toeplitz(col)
    A.append(mx)

for m in range(1, 2*d + 1):
    col = np.zeros(2*d + 1, dtype=complex)
    col[m] = -1j
    mx = toeplitz(col)
    A.append(mx)

status_LMI, optimal_value_LMI, optimal_vars_LMI = solve_LMI(A_0, A, c, verbose=False)

print("Toeplitz LMI Status:", status_LMI)
print("Toeplitz LMI Optimal Value:", optimal_value_LMI)
print("Toeplitz LMI Optimal Variables:", [var.value for var in optimal_vars_LMI])

In [5]:
# SPARSE LMI formulation for Sketchy CGAL
diag = np.ones(2*d + 1, dtype=complex)
A_0 = diags(diag)

A = []

for m in range(1, 2*d + 1):
    diag = np.ones(2*d + 1 - m, dtype=complex)
    mx = diags([diag, diag],offsets=[-m,m])
    A.append(mx)

for m in range(1, 2*d + 1):
    diag = np.ones(2*d + 1 - m, dtype=complex)
    mx = diags([-1j*diag, 1j*diag],offsets=[-m,m])
    A.append(mx)

A_norm = 2*d

U, Lambda, objective, Omega, z, y, S = run_solver(A_0, A, c, 2*d + 1, 4*d,
                                                    alpha=2*np.linalg.norm(c,ord=2), A_norm=A_norm,
                                                    R=int(np.ceil(np.log(d + 0.1))), T=10000,
                                                    trace_mode='bound')


print("Calculated Objective:", -objective)

Cholesky decomposition failed on the following matrix:
Calculated Objective: -0.0


In [ ]:
# print(c)
# print(int(np.ceil(np.log(d + 0.1))))
print(np.linalg.norm(c,ord=2))
print(np.linalg.norm(c,ord=1))